Funcion: get_features_num_regression


Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de regresión, es decir debe ser una variable numérica continua o discreta pero con alta cardinalidad, además de un argumento 'umbral_corr', de tipo float que debe estar entre 0 y 1 y una variable float "pvalue" cuyo valor debe ser por defecto "None".

La función debe devolver una lista con las columnas numéricas del dataframe cuya correlación con la columna designada por "target_col" sea superior en valor absoluto al valor dado por "umbral_corr". Además si la variable "pvalue" es distinta de None, sólo devolvera las columnas numéricas cuya correlación supere el valor indicado y además supere el test de hipótesis con significación mayor o igual a 1-pvalue.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable numérica continua del dataframe.

In [7]:
import pandas as pd
from scipy.stats import pearsonr

import pandas as pd
import numpy as np




def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Devuelve las columnas numéricas de un DataFrame cuya correlación con una
    columna objetivo supera un umbral definido, y opcionalmente un test de significación.

    Argumentos:
    df (pd.DataFrame): DataFrame que contiene los datos.
    target_col (str): Nombre de la columna objetivo (variable dependiente).
                      Debe ser numérica y de alta cardinalidad.
    umbral_corr (float): Valor entre 0 y 1 que indica el umbral mínimo de
                         correlación absoluta.
    pvalue (float, opcional): Nivel de significación estadística. Si es distinto
                              de None, se filtran solo las variables cuya
                              significación sea mayor o igual a 1 - pvalue.

    Devuelve:
    list: Lista de nombres de columnas numéricas que cumplen las condiciones.
          Devuelve None si alguna comprobación falla.
    """

    # ---------- COMPROBACIONES DE ENTRADA ----------

    # Comprobar que df es un DataFrame
    if not isinstance(df, pd.DataFrame):
        print("Error: df debe ser un pandas DataFrame.")
        return None

    # Comprobar que target_col existe en el DataFrame
    if target_col not in df.columns:
        print("Error: target_col no existe en el DataFrame.")
        return None

    # Comprobar que target_col es numérica
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        print("Error: target_col debe ser una variable numérica.")
        return None

    # Comprobar que umbral_corr es float entre 0 y 1
    if not isinstance(umbral_corr, float) or not (0 <= umbral_corr <= 1):
        print("Error: umbral_corr debe ser un float entre 0 y 1.")
        return None

    # Comprobar pvalue si se ha introducido
    if pvalue is not None:
        if not isinstance(pvalue, float) or not (0 < pvalue < 1):
            print("Error: pvalue debe ser None o un float entre 0 y 1.")
            return None

    # ---------- SELECCIÓN DE COLUMNAS NUMÉRICAS ----------

    columnas_numericas = df.select_dtypes(include="number").columns

    # Eliminar la columna target de la lista
    columnas_numericas = columnas_numericas.drop(target_col)

    # ---------- CÁLCULO DE CORRELACIONES ----------

    columnas_seleccionadas = []

    for col in columnas_numericas:
        # Eliminar filas con valores nulos
        data = df[[target_col, col]].dropna()

        # Necesitamos al menos 2 valores para correlación
        if len(data) < 2:
            continue

        corr, p_val = pearsonr(data[target_col], data[col])

        # Comprobar umbral de correlación
        if abs(corr) >= umbral_corr:
            # Si no se usa pvalue, se añade directamente
            if pvalue is None:
                columnas_seleccionadas.append(col)
            else:
                # Comprobar significación estadística
                if p_val <= pvalue:
                    columnas_seleccionadas.append(col)

    return columnas_seleccionadas
